In [33]:
!pip install fastapi uvicorn pyngrok streamlit transformers torch requests

In [34]:
%mkdir -p /content/chatbot_app
%cd /content/chatbot_app

/content/chatbot_app


In [35]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

app = FastAPI(title="Chatbot API")

model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

chat_histories = {}

class ChatRequest(BaseModel):
    user_id: str
    message: str

@app.post("/chat/")
def chat(request: ChatRequest):
    user_id = request.user_id
    message = request.message
    chat_history = chat_histories.get(user_id)
    new_input_ids = tokenizer.encode(message + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history, new_input_ids], dim=-1) if chat_history is not None else new_input_ids
    chat_history = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    chat_histories[user_id] = chat_history
    bot_message = tokenizer.decode(chat_history[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return {"reply": bot_message}

Overwriting main.py


In [36]:
%%writefile app.py
import streamlit as st
import requests
import uuid

API_URL = "http://127.0.0.1:8000/chat/"

if "user_id" not in st.session_state:
    st.session_state.user_id = str(uuid.uuid4())
st.title("Team 1 Chatbot Application")
if "messages" not in st.session_state:
    st.session_state.messages = []
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])
if prompt := st.chat_input("Type your message..."):
    st.chat_message("user").markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})
    payload = {"user_id": st.session_state.user_id, "message": prompt}
    response = requests.post(API_URL, json=payload)
    if response.status_code == 200:
        bot_reply = response.json()["reply"]
    else:
        bot_reply = "Server error. Please try again."
    st.chat_message("assistant").markdown(bot_reply)
    st.session_state.messages.append({"role": "assistant", "content": bot_reply})

Overwriting app.py


In [38]:
from pyngrok import ngrok
import threading, time, subprocess
ngrok.set_auth_token("34kVUW7lw88K5Oy5vwKhTRhlyT5_6rqGB2Lo1269LEm96wQDY") #replace this token with your own
def start_fastapi():
    subprocess.run(["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"])
threading.Thread(target=start_fastapi, daemon=True).start()
time.sleep(5)
public_url = ngrok.connect(8000).public_url
print("FastAPI public URL:", public_url)

FastAPI public URL: https://arden-meritorious-haggishly.ngrok-free.dev


In [39]:
def start_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501"])
threading.Thread(target=start_streamlit, daemon=True).start()
time.sleep(5)
streamlit_url = ngrok.connect(8501).public_url
print("Streamlit public URL:", streamlit_url)

Streamlit public URL: https://arden-meritorious-haggishly.ngrok-free.dev
